In [ ]:
'''%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
carrier_count = cat_df_flights['carrier'].value_counts()
sns.set(style="darkgrid")
sns.barplot(carrier_count.index, carrier_count.values, alpha=0.9)
plt.title('Frequency Distribution of Carriers')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Carrier', fontsize=12)
plt.show()'''

In [6]:
!pip install scikit-image

     |████████████████████████████████| 26.2MB 12.0MB/s eta 0:00:01   |█▊                              | 1.4MB 1.4MB/s eta 0:00:18     |█████▍                          | 4.4MB 1.4MB/s eta 0:00:16     |███████▉                        | 6.4MB 1.4MB/s eta 0:00:14     |████████████▏                   | 9.9MB 9.0MB/s eta 0:00:02     |████████████████                | 13.0MB 9.0MB/s eta 0:00:02     |███████████████████████▏        | 19.0MB 9.0MB/s eta 0:00:01     |█████████████████████████▌      | 20.8MB 12.0MB/s eta 0:00:01     |█████████████████████████▉      | 21.2MB 12.0MB/s eta 0:00:01     |██████████████████████████▎     | 21.5MB 12.0MB/s eta 0:00:01     |██████████████████████████▊     | 21.9MB 12.0MB/s eta 0:00:01     |█████████████████████████████▏  | 23.9MB 12.0MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 11.0MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 8.4MB/s eta 0:00:01     |█████                           | 317kB 8.4MB/s eta 0:00:01     |████

In [7]:
import pandas as pd
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

from tensorflow.keras import datasets, layers, models, regularizers, initializers
from skimage.io import imshow
from skimage.transform import rotate
from skimage.filters.edges import convolve

from skimage.io import imshow, imread
from skimage.transform import resize
from skimage.color import rgb2gray

np.random.seed(33)

In [8]:
df = pd.read_csv('data/labels_df.csv')

In [ ]:
loaded = np.load('data/image_array.npz')
all_img = loaded['a']
bw_img = loaded['b']

In [10]:
df.masterCategory.value_counts()

Apparel           21341
Accessories       11004
Footwear           9119
Personal Care      2393
Free Items          104
Sporting Goods       25
Home                  1
Name: masterCategory, dtype: int64

In [11]:
labels = df.masterCategory.copy()

In [18]:
labels.astype('category')

AttributeError: Can only use .cat accessor with a 'category' dtype

In [15]:
masterCat_list = list(labels.value_counts().index)

In [17]:
labels_dict = {}
for i in range(len(masterCat_list)):
    labels_dict[masterCat_list[i]] = i
labels_dict

{'Accessories': 1,
 'Apparel': 0,
 'Footwear': 2,
 'Free Items': 4,
 'Home': 6,
 'Personal Care': 3,
 'Sporting Goods': 5}

In [ ]:
#shuffle images and target var
i = np.random.choice( np.arange(len(images)), size=len(images), replace=False)
images = [images[ix] for ix in i]
y = y[i]